In [16]:
from model import load_model
from image_utils import imagenet_utils
import numpy as np
import os
from explanation_models import gradcam,gradcam_robust,gradcam_plusplus,guided_backprop
from tensorflow import keras
from tqdm import tqdm

In [17]:
folder='ILSVRC2012_img_val/'

test=load_model('VGG16')
model=test.model
input_dim=test.input_dim
layer_name='block5_conv3'
g= gradcam(model,layer_name,input_dim)


In [18]:
dict={}

In [20]:
for image_name in tqdm(os.listdir(folder)):
    img_path=os.path.join(folder,image_name)
    img = keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    preprocess_input =keras.applications.vgg16.preprocess_input
    # decode_predictions = keras.applications.vgg16.decode_predictions
    preprocessed_input=preprocess_input(array)
    heatmap=g.get_heatmap(preprocessed_input)
    dict[image_name]=heatmap


100%|██████████| 20/20 [00:22<00:00,  1.13s/it]


{'ILSVRC2012_val_00000001.JPEG': <tf.Tensor: shape=(14, 14), dtype=float32, numpy=
 array([[0.4492926 , 0.17348707, 0.        , 0.        , 0.04420624,
         0.14310774, 0.1989408 , 0.19192913, 0.18020889, 0.06442314,
         0.01115639, 0.03018879, 0.14039037, 0.44096315],
        [0.08289375, 0.02481465, 0.00424716, 0.02522944, 0.056704  ,
         0.11497172, 0.25245857, 0.21918036, 0.19657053, 0.06470162,
         0.0465047 , 0.05209708, 0.06815638, 0.4302483 ],
        [0.19938527, 0.11264212, 0.08030487, 0.11067336, 0.17402191,
         0.3210063 , 0.43211052, 0.35891846, 0.31166595, 0.22448845,
         0.08691055, 0.03800922, 0.15387192, 0.57808596],
        [0.17539573, 0.13341083, 0.161376  , 0.19948414, 0.12492727,
         0.02522208, 0.        , 0.        , 0.        , 0.15357034,
         0.2544628 , 0.24064878, 0.14077054, 0.623912  ],
        [0.31346786, 0.24402177, 0.18676369, 0.41337729, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.21435